In [1]:
import pandas as pd
import numpy as np

In [2]:
vistas = pd.read_csv('/home/luciano/orga-datos/datos_filtrados/vistas.csv')
vistas_np = pd.read_csv('/home/luciano/orga-datos/datos_filtrados/vistas_np.csv')

In [3]:
kaggle = pd.read_csv('/home/luciano/orga-datos/test_final_100k.csv')

In [5]:
import preprocesar_set.datasets.postulantes as postulantes

Cargando postulantes...OK


In [6]:
df = pd.merge(kaggle, postulantes.df_genero_edad, how='left', left_on='idpostulante', right_index=True)
df = pd.merge(df, postulantes.df_educacion, how='left', left_on='idpostulante', right_index=True)

df['hay_informacion_basica'] = 1
df[df['sexo'].isna() | df['edad'].isna() | df['nivel_educativo'].isna()] = 0
df['sexo'] = df['sexo'].fillna(0)
df['edad'] = df['edad'].fillna(0)
df['nivel_educativo'] = df['nivel_educativo'].fillna(0)

s = df.rename(columns={'sexo': 'sexo_postulante', 'edad': 'edad_postulante', 
        'nivel_educativo':'nivel_educativo_postulante'})

In [7]:
import preprocesar_set.datasets.postulaciones as postulaciones

Cargando postulaciones...OK


In [8]:

df_data = pd.merge(df[['idaviso']].drop_duplicates(), postulaciones.df, on='idaviso', how='inner')
df_data = pd.merge(df_data, postulantes.df_genero_edad, on='idpostulante', how='right')
df_data = pd.merge(df_data, postulantes.df_educacion, on='idpostulante', how='right')
df_data = df_data.drop(axis=1, labels=['idpostulante'])\
    .rename(columns={'edad': 'edad_media', 'sexo':'sexo_medio', 
        'nivel_educativo': 'nivel_educativo_medio'})
df_data = df_data.groupby('idaviso').agg('mean').reset_index()
df_data = pd.merge(df, df_data, on='idaviso', how='left')
df_data['hay_informacion_media_aviso'] = 1
df_data[df_data['edad_media'].isna() | df_data['sexo_medio'].isna() | df_data['nivel_educativo_medio'].isna()] = 0

In [ ]:
df = kaggle
FACTOR = 2
EDAD_MEDIA = 30

df_data = pd.merge(df[['idaviso']].drop_duplicates(), postulaciones.df, on='idaviso', how='inner')
df_data = pd.merge(df_data, postulantes.df_genero_edad, on='idpostulante', how='right')
df_data = pd.merge(df_data, postulantes.df_educacion, on='idpostulante', how='right')
df_data = df_data.drop(axis=1, labels=['idpostulante'])\
    .rename(columns={'edad': 'edad_media', 'sexo':'sexo_medio', 
        'nivel_educativo': 'nivel_educativo_medio'})

df_data_2 = df_data.groupby('idaviso')['edad_media'].agg(lambda x: (np.sum(x) + FACTOR * EDAD_MEDIA) / (FACTOR + len(x)))
df_data = pd.merge(df, df_data, on='idaviso', how='left')
df_data['hay_informacion_media_aviso'] = 1
df_data[df_data['edad_media'].isna() | df_data['sexo_medio'].isna() | df_data['nivel_educativo_medio'].isna()] = 0